#### Abschnitt 1: Inverted Index mit Ausgabe der TFIDF-Werte

In [ ]:
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm


#definieren der funktion für den inverted index
def inverted_index(documents):
    inverted_index = {}

    for document in documents:
        text = document["text"]
        document_name = document["name"]

        # text tokenisieren über re.findall, zahlen im text werden nicht berücksichtigt
        terms = re.findall(r'\w+', text.lower())

        # den tfidfVectorizer über sklearn festlegen zur berechnung
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([text])

        # terme und ihre tf-idf-werte berechnen lassen
        # matrix in ein array umwandeln, zugriff auf immer ein dokument
        terms = vectorizer.get_feature_names_out()
        tfidf_values = tfidf_matrix.toarray()[0]

        # dokument, terme und tf-idf-werte zusammenfügen beim inverted index für spätere ausgabe
        for term, tfidf in zip(terms, tfidf_values):
            if term.isdigit():  # überprüfung, ob der term eine zahl ist, falls ja: überspringen
                continue
            if term not in inverted_index:
                inverted_index[term] = []
            inverted_index[term].append((document_name, tfidf))

    return inverted_index

# pfad einfügen lassen und leere liste erstellen zum einlesen
path2corpus = input("Bitte den genauen Pfad hier einfügen...")
documents = []

# einlesen der dokumente über name und text in form eines dictionary
for file_name in tqdm(os.listdir(path2corpus), desc="Einlesen der Dokumente..."):
    with open(os.path.join(path2corpus, file_name), "r", encoding="utf-8") as file:
        document = {
            "name": file_name,
            "text": file.read()
        }
        documents.append(document)
        

#inverted-index-funktion auf die dokumente anwenden
inverted_index = inverted_index(documents)

        
#gibt den inverted index in der endausgabe aus mit term übergeordnet und darunter...
#...die auflistung der jeweiligen dateinamen und des tfidf
for term, key in inverted_index.items():
    print(f"Term: {term}")
    for key in key:
        document_name, tfidf = key
        print(f"\tDateiname: {document_name}, TFIDF: {tfidf}")
    print()


#### Abschnitt 2: Vektorraum-Modell über Kosinus-Ähnlichkeiten der TFIDF-Werte

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
from tqdm import tqdm

#pfad zum verzeichnis mit den Dokumenten
path2corpus = input("Bitte geben Sie den Pfad zum Verzeichnis mit den Dokumenten ein: ")

#dokumente einlesen
documents = []
file_names = []
for file_name in tqdm(os.listdir(path2corpus), desc="Einlesen der Dokumente"):
    with open(os.path.join(path2corpus, file_name), "r", encoding="utf-8") as file:
        document = file.read()
        documents.append(document)
        file_names.append(file_name)

#tfidfVectorizer über von sklearn definieren
vectorizer = TfidfVectorizer()

# dokumentdaten in eine tfidf-matrix umwandeln
tfidf_matrix = vectorizer.fit_transform(documents)

#feature_names berechnen lassen (vocabulary)
vocabulary = vectorizer.get_feature_names_out()


#wortsuche über while-schleife durchführen
while True:
    input_word = input("Finde folgendes Wort: ")
    if not input_word:
        print("Kein Wort eingegeben. Das Programm wird abgebrochen.")
        break
    else:
        #input_word = input_word.split()    
    
        #vektor für eingabewort berechnen
        base_vector = vectorizer.transform([input_word])

        #tfidf-werte für das eingabewort abrufen
        base_tfidf = base_vector.toarray()[0]

        #cosinus-ahnlichkeit zwischen eingabewortvektor und dokumentvektoren berechnen
        cosine_sim = cosine_similarity(base_vector, tfidf_matrix)

        #ähnlichkeitswerte für das eingabewort für alle dokumente bekommen
        similarity_scores = cosine_sim.flatten()

        #indizes der nicht-null ähnlichkeitswerte erhalten, um 0er auszufiltern
        filter_zeroes = np.nonzero(similarity_scores)

        #filtern der ähnlichkeitswerte, tfidf-matrix und dateinamen
        filter_cosine = similarity_scores[filter_zeroes]
        filter_filenames = np.array(file_names)[filter_zeroes]

        #gefilterte kosinus-ähnlichkeiten absteigend sortieren mithilfe von umgekehrtem slicing [::-1]
        sorted_indices = np.argsort(filter_cosine)[::-1]

        #10 häufigste nonzero kosinus-ähnlichkeitswerte und summe der entsprechenden tfidf-werte ausgeben
        if len(sorted_indices) == 0:
            print("Wort nicht im Korpus enthalten.\n") 
        else:    
            print("10 häufigste TF-IDF-Werte, berechnet über die Kosinus-Ähnlichkeit:")
            for index in sorted_indices[:10]:
                file_name = filter_filenames[index]
                similarity = filter_cosine[index]
                if similarity != [0.]: #nicht-0-werte definieren für ausgabe
                    print(f'Dokument: {file_name}, Cos-Ähnlichkeit: {similarity}\n')
                else:
                    continue

requirements.txt nicht als Abgabe möglich wegen Abgabebegrenzung in WueCampus, daher hier als Kommentar eingefügt:

- numpy==1.23.5
- tqdm==4.65.0
- scikit-learn==1.1.3
- re==2.2.1
- pandas=2.0.0